<a href="https://colab.research.google.com/github/prajwalnayak7/llama-hack/blob/main/Cantonese_data_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=9724b2c9ac7d8fa3f311dd81cdc94c0fac30976774bee5136918da2f87533ab9
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succ

In [57]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.4 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/meta-llama/llama-recipes.git

Cloning into 'llama-recipes'...
remote: Enumerating objects: 11253, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 11253 (delta 0), reused 4 (delta 0), pack-reused 11239 (from 1)
Receiving objects: 100% (11253/11253), 110.36 MiB | 25.22 MiB/s, done.
Resolving deltas: 100% (6929/6929), done.


In [ ]:
import zipfile

# Define the path to the zip file
zip_file_path = '/content/transcripts.json.zip'  # Update this path

# Step 1: Unzip the file and read the JSON data
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')  # Specify extraction path

# Assuming the JSON file is named 'transcripts.json' after extraction
json_file_path = '/content/transcripts.json'  # Update this path if necessary


In [ ]:
import json
import re

with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

def parse_conversations(data):
    conversations = []

    for convo_id, text in data.items():
        # Detect language based on the presence of Arabic characters
        language = "ar" if any("\u0600" <= char <= "\u06FF" for char in text) else "en"

        # Initialize conversation structure
        structured_convo = {"id": convo_id, "language": language, "turns": []}

        # Split by alternating "D:" and "P:" tags using regex
        segments = re.split(r"(?<=\n)(D: |P: )", text)  # Keeps labels in the split

        # Temp variables to store current turn text for doctor and patient
        current_doctor_text = ""
        current_patient_text = ""

        # Iterate through segments to organize turns
        for i in range(1, len(segments), 2):  # Start from the first tag
            speaker = segments[i].strip()  # Get speaker tag (D: or P:)
            text_segment = segments[i + 1].strip()  # Get associated text

            if speaker == "D:":
                # Store any existing doctor-patient turn before starting a new turn
                if current_doctor_text or current_patient_text:
                    structured_convo["turns"].append({
                        "doctor": current_doctor_text,
                        "patient": current_patient_text
                    })
                    current_patient_text = ""  # Reset patient text for new turn
                current_doctor_text = text_segment  # Set new doctor text

            elif speaker == "P:":
                current_patient_text = text_segment  # Set patient text

        # Append the last collected doctor-patient turn, if any
        if current_doctor_text or current_patient_text:
            structured_convo["turns"].append({
                "doctor": current_doctor_text,
                "patient": current_patient_text
            })

        conversations.append(structured_convo)

    return conversations

# Parse the conversations using the updated function
conversations = parse_conversations(data)

# Display parsed data for verification
for convo in conversations[:3]:  # Display first 3 conversations
    print(json.dumps(convo, indent=2, ensure_ascii=False))

{
  "id": "2055",
  "language": "en",
  "turns": []
}
{
  "id": "291",
  "language": "en",
  "turns": []
}
{
  "id": "102",
  "language": "en",
  "turns": [
    {
      "doctor": "",
      "patient": "Good morning, Doctor. I've been experiencing some strange symptoms lately."
    },
    {
      "doctor": "Can you tell me what kind of symptoms you're experiencing?",
      "patient": "Sure, I've been feeling dizzy and unsteady on my feet, and I've also been having headaches."
    },
    {
      "doctor": "Interesting. Based on your age and the symptoms you're experiencing, it could be hypertension.",
      "patient": "Hypertension? What's that?"
    },
    {
      "doctor": "Hypertension, also known as high blood pressure, is when the force of blood against your arteries is consistently too high.",
      "patient": "Oh, I see. And what can I do about it?"
    },
    {
      "doctor": "To manage hypertension, I recommend that you practice meditation, take salt baths, reduce stress, and ma

In [ ]:
conversations

[{'id': '2055', 'language': 'en', 'turns': []},
 {'id': '291', 'language': 'en', 'turns': []},
 {'id': '102',
  'language': 'en',
  'turns': [{'doctor': '',
    'patient': "Good morning, Doctor. I've been experiencing some strange symptoms lately."},
   {'doctor': "Can you tell me what kind of symptoms you're experiencing?",
    'patient': "Sure, I've been feeling dizzy and unsteady on my feet, and I've also been having headaches."},
   {'doctor': "Interesting. Based on your age and the symptoms you're experiencing, it could be hypertension.",
    'patient': "Hypertension? What's that?"},
   {'doctor': 'Hypertension, also known as high blood pressure, is when the force of blood against your arteries is consistently too high.',
    'patient': 'Oh, I see. And what can I do about it?'},
   {'doctor': "To manage hypertension, I recommend that you practice meditation, take salt baths, reduce stress, and make sure you're getting proper sleep. In addition, it's important that you monitor your

In [53]:
!pip install frontend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.1 MB/s eta 0:00:00


In [55]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 64.5 MB/s eta 0:00:00


In [49]:
import zipfile

# Define the path to the zip file
zip_file_path = '/content/dataset.zip'  # Update this path

# Step 1: Unzip the file and read the JSON data
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')  # Specify extraction path

# Assuming the JSON file is named 'transcripts.json' after extraction
file_path = '/content/dataset'  # Update this path if necessary

In [60]:
import pandas as pd
import pdfplumber  # Alternative library for reading PDFs
import os

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

def create_conversations_dataframe(pdf_folder_path):
    data = []
    conversation_id = 1  # Starting ID for conversations

    for filename in os.listdir(pdf_folder_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, filename)
            conversation_text = extract_text_from_pdf(pdf_path)
            data.append({"conversation_id": conversation_id, "conversation": conversation_text})
            conversation_id += 1

    df = pd.DataFrame(data)
    return df

# Usage:
pdf_folder_path = "/content/dataset "  # Your specified path
conversations_df = create_conversations_dataframe(pdf_folder_path)

# Display the DataFrame to verify
conversations_df.head()


,conversation_id,conversation
0,1,"{\n""conversation"": [\n{\n""speaker"": ""Nurse"",\n..."
1,2,"{\n""conversation"": [\n{\n""nurse"": ""Hi, I’m Nur..."
2,3,"""Role"",""Dialogue""\n""Nurse"",""Good evening! My n..."
3,4,"{\n""conversation"": [\n{\n""speaker"": ""Nurse"",\n..."
4,5,"""Nurse Question"",""Patient Response""\n""What bri..."


In [61]:
conversations_df

,conversation_id,conversation
0,1,"{\n""conversation"": [\n{\n""speaker"": ""Nurse"",\n..."
1,2,"{\n""conversation"": [\n{\n""nurse"": ""Hi, I’m Nur..."
2,3,"""Role"",""Dialogue""\n""Nurse"",""Good evening! My n..."
3,4,"{\n""conversation"": [\n{\n""speaker"": ""Nurse"",\n..."
4,5,"""Nurse Question"",""Patient Response""\n""What bri..."
5,6,"{\n""conversation"": [\n{\n""speaker"": ""Nurse"",\n..."
6,7,"""Nurse Question"",""Patient Response""\n""What bri..."
7,8,Certainly! Below is a scenario between a nurse...
8,9,```json\n{\n“conversation”: [\n{\n“speaker”: “...
9,10,"{\n""conversation"": [\n{\n""speaker"": ""Nurse"",\n..."


In [66]:
import pandas as pd
from googletrans import Translator

# Initialize the Translator
translator = Translator()

# Sample DataFrame (use your `conversations_df` instead)
# conversations_df = pd.DataFrame({
#     "conversation_id": [1, 2],
#     "conversation": ["Hello, how are you?", "I am fine, thank you!"]
# })

# Function to translate text
def translate_text(text, target_language="zh-TW"):
    try:
        translated = translator.translate(text, dest=target_language)
        return translated.text
    except Exception as e:
        print(f"Error translating text: {e}")
        return text  # Return original text if translation fails

# Apply the translation to the 'conversation' column
target_language = "zh-TW"  # Change to the language code you want (e.g., "fr" for French, "hi" for Hindi)
conversations_df["translated_conversation"] = conversations_df["conversation"].apply(lambda x: translate_text(x, target_language))

# Display the DataFrame with translated conversations
conversations_df.head()


,conversation_id,conversation,translated_conversation
0,1,"{\n""conversation"": [\n{\n""speaker"": ""Nurse"",\n...",{\n“對話”： [\n{\n“發言人”：“護士”，\n“消息”：“你好，我是護士艾米麗。我...
1,2,"{\n""conversation"": [\n{\n""nurse"": ""Hi, I’m Nur...",{\n“對話”： [\n{\n“護士”：“嗨，我是護士艾米麗（Emily）。我看到你已經從貓...
2,3,"""Role"",""Dialogue""\n""Nurse"",""Good evening! My n...",“角色”，“對話”\n“護士”，“晚上好！我叫莎拉，我是急診室的護士。我可以叫你的名字嗎？”...
3,4,"{\n""conversation"": [\n{\n""speaker"": ""Nurse"",\n...",{\n“對話”： [\n{\n“發言人”：“護士”，\n“消息”：“你好，我是護士艾米麗。我...
4,5,"""Nurse Question"",""Patient Response""\n""What bri...",“護士問題”，“患者反應”\n“是什麼把你帶到急診室？”，“過去幾個小時我一直在胃和腹部疼痛...


In [72]:
onversations_df = conversations_df.rename(columns={
    "conversation_id": "ID",
    "conversation": "Original Conversation",
    "translated_conversation": "Cantonese Conversation"
})

In [69]:
onversations_df.to_csv('cantonise_conversations.csv')

In [73]:
import pandas as pd
import json

# Convert DataFrame to JSONL format with Cantonese as input and English as output
jsonl_data = []
for _, row in onversations_df.iterrows():
    jsonl_data.append({
        "input": row["Cantonese Conversation"],  # Cantonese conversation as input
        "output": row["Original Conversation"]   # English conversation as target output
    })

# Save as a JSONL file
output_file = "cantonese_to_english.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"Data has been saved to {output_file}")


Data has been saved to cantonese_to_english.jsonl


In [75]:
import pandas as pd
import json

# Convert DataFrame to JSONL format with combined text field
jsonl_data = []
for _, row in onversations_df.iterrows():
    # Format: "Translate from Cantonese: <Cantonese text> to English: <English translation>"
    combined_text = f"Translate from Cantonese: {row['Cantonese Conversation']} to English: {row['Original Conversation']}"
    jsonl_data.append({"text": combined_text})

# Save as a JSONL file
output_file = "cantonese_to_english_translation.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"Data has been saved to {output_file}")

Data has been saved to cantonese_to_english_translation.jsonl
